# Memory

Describing memory and getting it synthesized correctly is cumbersome.
Therefore, this will only demonstrate a simple approach to generate memory.
Using [Signal arrays](arrays.ipynb), a simple `r1w1` simplex dual port memory can be inferred to HDL.
This modification enables shared variable output for VHDL to support true dual port behaviour.
The general strategy with the MyIRL synthesis is to use a blackbox element (rely on the library)

In [1]:
import sys
sys.path.insert(0, "/home/testing/src/myhdl2")

from myhdl import intbv
from myirl import *
import random

## Dual port memories

We can not use a `SigArray` for the RAM, because a true dual port RAM would provoke unresolved multiple drivers.
Therefore we hack the `myirl.lists.SigArray` class by deriving and choosing a different element in return of the `__getitem__` member which uses a shared variable.

**Note** Not working in VHDL-2008

In [2]:
from myirl import lists
from myirl.kernel import extensions, utils

from myirl import targets

class CellAssign:  # extensions.ElemAssign
    def __init__(self, parent, portion, val, width):
        self._vref = parent
        self.portion = portion
        self.value = val
        self.width = width
        
    def emit(self, context):
        tgt = context.target
        n, v = self._vref.identifier, self.value
        p = self.portion
        sz = self.width
        if tgt.lang == 'VHDL':
            context.output("%s(to_integer(%s)) := %s;\n" % (n, p, base.convert(v, tgt, sz)))        
        elif tgt.lang == 'Verilog':
            context.output("%s[%s] <= %s;\n" % (n, p, base.convert(v, tgt, sz)))           
        else:
            raise TypeError("Unsupported target %s" % type(tgt))


    def get_sources(self, srcs):
        if isinstance(self.value, base.Sig):
            self.value.get_sources(srcs)
    
    def get_drivers(self, drvs):
        pass
    
t
class RamBuffer(lists.SigArray):    
    def __getitem__(self, item):
        if isinstance(item, (Sig, int)):
            # We can not just return self.val[item], as the iterator
            # has not initialized yet.
            return ArrayElem(self, item)
        else:
            raise TypeError("Multi item slicing of iterator not supported")

Then define a RAM port (legacy class constructs will suffice) and the actual RAM implementation:

In [3]:
class RamPort:
    _inputs = ['we', 'wa', 'ra', 'wd']
    _outputs = ['rd']
    _other = ['clk']
    def __init__(self, AWIDTH, DWIDTH):
        self.clk = ClkSignal()
        self.we = Signal(bool())
        self.ra, self.wa = [ Signal(intbv()[AWIDTH:]) for i in range(2) ]
        self.rd, self.wd = [ Signal(intbv()[DWIDTH:]) for i in range(2) ] 
        
@block
def tdp_ram(pa, pb, INITDATA):
    inst = []
 
    def gen_logic(p, i):
        "Generate port mechanics inline"
        
        rd = p.rd.clone("rd%d" % i)
        
        @genprocess(p.clk, EDGE=p.clk.POS)
        def proc():
            yield [
                proc.If(p.we == True).Then(
                    buf[p.wa].set(p.wd)
                ),
                rd.set(buf[p.ra])
            ]
        proc.rename("proc%d" % i)
        wirings = [
            p.rd.wireup(rd)
        ]
        return proc, wirings

    buf = RamBuffer(INITDATA)
    
    for i, p in enumerate([pa, pb]):
        inst += (gen_logic(p, i))
    
    return inst

### Basic checks

Verify we can evaluate the content:

In [4]:
a = RamBuffer([intbv(i + 0x80)[8:] for i in range(200) ])
b = Signal(intbv(10)[5:])
a[b].evaluate()

intbv(138)

In [5]:
a[0].wire

intbv(128)

In [6]:
from myirl import targets
from myirl.test.common_test import run_ghdl, run_icarus

random.seed(0)

def test_vhdl():
    RAM_CONTENT = [ intbv(random.randint(0, 2 ** 9))[8:] for i in range(2 ** 9) ]
    pa, pb = [ RamPort(AWIDTH=9, DWIDTH=8) for i in range(2) ]
    inst = tdp_ram(pa, pb, RAM_CONTENT)
    
    f = inst.elab(targets.VHDL)
    run_ghdl(f, inst, std = "93", debug = True) # Note we run with std '93'
    return f

In [7]:
f = test_vhdl()

Adding INITDATA to port dict (fallback)
 Writing 'tdp_ram' to file /tmp/myirl_tdp_ram__tfeq0gj/tdp_ram.vhdl 


In [9]:
! grep -A 20 MyIRL {f[0]}

architecture MyIRL of tdp_ram is
    -- Local type declarations
    -- Signal declarations
    signal s_dd84 : unsigned(7 downto 0);
    type a_v_f773 is array (0 to 511) of unsigned(7 downto 0);
    shared variable v_f773 : a_v_f773;
    signal s_ce16 : unsigned(7 downto 0);
begin
    
proc0:
    process(pa_clk)
    begin
        if rising_edge(pa_clk) then
            if (pa_we = '1') then
                v_f773(to_integer(pa_wa)) := pa_wd;
            end if;
            s_dd84 <= v_f773(to_integer(pa_ra));
        end if;
    end process;
    pa_rd <= s_dd84;
    
--
end architecture MyIRL;



## Verilog variant

The same design translated to Verilog, run through a syntax check only.

In [8]:
def test_verilog():
    RAM_CONTENT = [ intbv(random.randint(0, 2 ** 9))[8:] for i in range(2 ** 9) ]
    pa, pb = [ RamPort(AWIDTH=9, DWIDTH=8) for i in range(2) ]
    inst = tdp_ram(pa, pb, RAM_CONTENT)
    
    f = inst.elab(targets.Verilog)
    run_icarus(f, inst, debug = True)
    return f

In [10]:
f = test_verilog()

Adding INITDATA to port dict (fallback)
 Module tdp_ram: Existing instance tdp_ram, rename to tdp_ram_1 
 Writing 'tdp_ram_1' to file /tmp/myirl_tdp_ram_krcneh2w/tdp_ram_1.v 
DEBUG SKIP INST [pa_rd <= s_8a50] <class 'list'>
DEBUG SKIP INST [pb_rd <= s_9a6a] <class 'list'>


In [11]:
!cat {f[0]}

// File generated from source:
//     /tmp/ipykernel_2176/330155752.py
// (c) 2016-2022 section5.ch
// Modifications may be lost, edit the source file instead.

`timescale 1 ns / 1 ps
`include "aux.v"

module tdp_ram_1
    (
        input  pa_clk,
        input  pa_we,
        input [8:0] pa_ra,
        input [8:0] pa_wa,
        output [7:0] pa_rd,
        input [7:0] pa_wd,
        input  pb_clk,
        input  pb_we,
        input [8:0] pb_ra,
        input [8:0] pb_wa,
        output [7:0] pb_rd,
        input [7:0] pb_wd
    );
    // Local type declarations
    // Signal declarations
    reg [7:0] s_8a50;
    // Dummy array definition v_8c25
    reg [7:0] v_8c25[0:511];
    reg [7:0] s_9a6a;
    always @ (posedge pa_clk ) begin : PROC0
        if ((pa_we == 1'b1)) begin
            v_8c25[pa_wa] <= pa_wd;
        end
        s_8a50 <= v_8c25[pa_ra];
    end
    assign pa_rd = s_8a50;
    always @ (posedge pb_clk ) begin : PROC1
        if ((pb_we == 1'b1)) begin
            v_8c2